In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datetime as dt
import time

from typing import Dict
from session import VevenSession


async def mass_get_cookies(credentials: Dict[str, str], event_url):
    sessions = {}
    for un, pw in credentials.items():
        print('\nUser:', un)
        Session = VevenSession(
            event_url=event_url,
            username=un, password=pw
        )
        await Session.get_cookies()
        sessions[un] = Session
    return sessions


def timed_mass_register(trigger, sessions):
    while dt.datetime.now() < trigger:
        time.sleep(0.001)
    
    responses = {}
    print('\nStarting registers. Time:', dt.datetime.now())
    for un, Session in sessions.items():
        t0 = time.time()
        res = Session.register()
        if res.status_code == 201:
            print('{} registered at appx. {}\nTime elapsed: {}s'.format(
                un, dt.datetime.now(), (time.time() - t0)))
        else:
            res = Session.register()
            print('{} failed to register. Retry code: {}\nTime elapsed: {}s'.format(
                un, res.status_code, (time.time() - t0)))
        responses[Session.username] = res
        
    return responses

In [3]:
event_url = 'https://omega.ntnu.no/events/show/522'
trigger = dt.datetime(year=2020, month=9, day=15, hour=12)
credentials = {
    'markskag': '',
    'larsgmu' : '',
    'knutve' : '',
    '': '',
}

# Register all users in credentials to event
sessions = await mass_get_cookies(credentials, event_url)
responses = timed_mass_register(trigger, sessions)

# Retry if some of the requests failed
failed_req_users = [user for user, res in responses.items() if res.status_code != 201]
if len(failed_req_users):
    print('Retrying for users:', failed_req_users)
    failed_sessions = {un: Session for un, Session in sessions.items() if un in failed_req_users}
    timed_mass_register(trigger, failed_sessions)


User: markskag
Fetching new cookies with webdriver.
New cookies fetched.
Valid cookies: True

User: larsgmu
Fetching new cookies with webdriver.
New cookies fetched.
Valid cookies: True

User: knutve
Fetching new cookies with webdriver.
New cookies fetched.
Valid cookies: True

Starting registers. Time: 2020-09-10 11:16:00.000276
markskag failed to register. Retry code: 400
Time elapsed: 0.21846580505371094s
larsgmu registered at appx. 2020-09-10 11:16:00.370452
Time elapsed: 0.15013623237609863s
knutve registered at appx. 2020-09-10 11:16:00.574011
Time elapsed: 0.20333480834960938s
Retrying for users: ['markskag']

Starting registers. Time: 2020-09-10 11:16:00.575894
markskag failed to register. Retry code: 400
Time elapsed: 0.25483059883117676s
